## Natural Language Processing - Summer Term 2023
### Hochschule Karlsruhe
### Lecturer: Prof. Dr. Jannik Strötgen
### Thanks to: Jun.-Prof. Dr. Andreas Spitz and his tutors.

# Exercise 06

### You will learn about:

- Text Similarity
- Levenshtein (Edit) Distance
- TF-IDF Vectors and Cosine Similarity

---

### Remark:

- You have **two weeks** until we discuss the solutions.

---


## Task 1 - Levenshtein (Edit) Distance (pen and paper) (10P):

### Part 1:
Calculate the Levenshtein distance between words 
- "signed" and "sealed"

Fill out the edit distance table like it is shown on the lecture slides. (In addition to the lecture, I will provide further information how to do that in more detail via ILIAS).

In [ ]:
def levenshteinDistance(str1, str2):
    m = len(str1)
    n = len(str2)
    d = [[i] for i in range(1, m + 1)]   # d matrix rows
    d.insert(0, list(range(0, n + 1)))   # d matrix columns
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if str1[i - 1] == str2[j - 1]:  
                substitutionCost = 0
            else:
                substitutionCost = 1
            d[i].insert(j, min(d[i - 1][j] + 1,
                               d[i][j - 1] + 1,
                               d[i - 1][j - 1] + substitutionCost))
            
    # printing the matrix
    for l in d:
        print(*l)

    return d[-1][-1]

In [ ]:
levenshteinDistance("signed","sealed")

---

### Part 2: 
What is the advantage and disadvantage of this method for measuring word similarity?

Vorteile:
- Sprachagnostisch, da die Levenshtein Distanz auf Buchstabenebene agiert, ist die Sprache irrelevant.
- Algorit
Nachteile:
- Semantik wird nicht betrachtet, Wörter die Synonyme sind, können große Distanz aufweisen.
- Komplexität O(n*m), bei langen Wörten suboptimal


---

## Task 2 - Text Similarity (10P):

### Part 1:
Your task is to calculate pair-wise Cosine similarities 
between the first 1000 sentences of the 'debates' dataset. 
Calculate the similarities on sentence TF-IDF vectors. Plot the similarity values in a heatmap visualization. 

- Calculate the Cosine similarity and visualize the results without cleaning the data.

- Calculate the Cosine similarity and visualize the results after cleaning the data (think of appropriate methods for data cleaning).

In [ ]:
# you can use the given libraries or choose different ones for the similarity calculation
# use plotly library to create a heatmap visualization

import json
import nltk
import plotly
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
plotly.offline.init_notebook_mode(connected=True)


In [ ]:
nltk.download('punkt')

In [ ]:
# read debates

with open('data/texts.json', 'r') as infile:
    data = json.load(infile)

content_debates = data['debates']

In [ ]:
from nltk.tokenize import sent_tokenize
corpus = sent_tokenize(content_debates)
corpus

In [ ]:
from nltk.tokenize import word_tokenize
def tokenize_sentence(sentence):
    return word_tokenize(sentence)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=tokenize_sentence,lowercase=False)
tfidf = vectorizer.fit_transform(corpus[:1000])
print(f"Features: {vectorizer.get_feature_names_out()[:500]}")
print(True if "the" in vectorizer.get_feature_names_out() else False)
print(f"Shape: {tfidf.shape}")
pairwise_similarity = tfidf * tfidf.T 
arr = pairwise_similarity.toarray() 

In [ ]:
fig1 = px.imshow(arr)
fig1

In [ ]:
import numpy as np

def get_descending_similar_sent(arr):
    # diagonal always 1 because of same sentence, replace with NaN
    
    # remove lower half of matrix and diagonal, to remove duplicates 
    arr[np.arange(arr.shape[0])[:,None] > np.arange(arr.shape[1])] = np.nan
    np.fill_diagonal(arr, np.nan)  
    
    n = arr.shape[1]
    sorted_arr = np.argsort(arr, axis=None)[::-1].__divmod__(n)
    # because of the nan values, sorting does not work as intented, when sorting descending NaN first values
    sorted_similarity = list(zip(*np.roll(sorted_arr,-np.count_nonzero(np.isnan(arr)))))
    return sorted_similarity


In [ ]:
desc_sim = get_descending_similar_sent(arr)
for x in desc_sim[:50]:
    print(corpus[x[0]],corpus[x[1]], x, arr[x[0]][x[1]])

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

stemmer = nltk.stem.snowball.SnowballStemmer("english")
# tokenize
tokenized_sentence  = [word_tokenize(t.translate(str.maketrans('','',string.punctuation))) for t in corpus]

# removing stop words
filter_tokens = [[word for word in sentence if word not in stop_words]for sentence in tokenized_sentence]

# stemming
stemmed_tokens = [[stemmer.stem(word) for word in sentence ]for sentence in filter_tokens]


In [ ]:
def identity_tokenizer(text):
    return text

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False)
tfidf = vectorizer.fit_transform(filter_tokens[:1000])
pairwise_similarity = tfidf * tfidf.T 
arr = pairwise_similarity.toarray() 

In [ ]:
fig2 = px.imshow(arr)

In [ ]:
desc_sim = get_descending_similar_sent(arr)
for x in desc_sim[:50]:
    print(corpus[x[0]],corpus[x[1]], x, filter_tokens[x[0]], filter_tokens[x[1]], arr[x[0]][x[1]])

In [ ]:
fig1

In [ ]:
fig2

---

#### Part 2:

- Interpret the results. What kind of insights can you gain from the heatmap visualizations?
- Ohne Säuberung der Daten mehr "Rauschen" in der Heatmap, gerade durch die Stopwords die eine hohe Häufigkeit haben.

- How does data cleaning influence the similarity results? 
- Es gibt weniger Ähnlichkeiten und durch das löschen der Stopwords + Stemming wird teilweise zu viel Kontext entfernt, wodurch Sätze eine Ählichkeit von 1 bekommen, aber unterschiedliche Aussagen enthalten.

<font color='ff000000'>\# TEXT SUBMISSION ANSWER HERE (Double click to edit) </font>

---


## Task 3 - Search Engine (10P):

### Part 1:
Read the critics for the first 200 movies from the 'rottentomatoes' dataset. 
First, create a document for each movie by concatenating their critic entries. 
Remove stop words, and compute TF-IDF vectors for each document.

In [ ]:
# TODO - ADD YOUR CODE HERE



### Part 2:
Write a function for querying the data. 
Given a set of input terms as parameter, the function should remove stop words in the input, 
compute a TF-IDF vector of the query, and match it to all stored document vectors by using Cosine similarity. 
Rank all documents by Cosine similarity to the query and output the 10 most similar documents to the query.


In [ ]:
# TODO - ADD YOUR CODE HERE


### Part 3:
Run a few queries on the data. Discuss the types of errors that your search engine makes.

<font color='ff000000'>\# TEXT SUBMISSION ANSWER HERE (Double click to edit) </font>

#### Submitting your results:

To submit your results, please:

- save this file, i.e., `ex??_assignment.ipynb`.
- if you reference any external files (e.g., images), please create a zip or rar archive and put the notebook files and all referenced files in there.
- login to ILIAS and submit the `*.ipynb` or archive for the corresponding assignment.

**Remarks:**
    
- Do not copy any code from the Internet. In case you want to use publicly available code, please, add the reference to the respective code snippet.
- Check your code compiles and executes, even after you have restarted the Kernel.
- Submit your written solutions and the coding exercises within the provided spaces and not otherwise.
- Write the names of your partner and your name in the top section.